# Importing Libraries

In [30]:
#Importing Libraries
import time
import ray
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

# Define functions to create SL and DL Models

In [59]:
# Define a function to create Shallow Learning Model (KNN)
def create_shallow_model():
    model = KNeighborsClassifier(n_neighbors=5)
    return model

# Define a function to create Deep Learning Model (DNN)
def create_deep_model(num_input, num_output):
    model = keras.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(num_input,)))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(num_output, activation='sigmoid'))  # Use softmax activation for multi-class classification
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Define Parallel Functions to train SL and DL

## Function to Train Shallow Model

In [32]:
@ray.remote
def train_shallow_model(X_train, y_train, X_test, y_test):
    # Create and train the Shallow Learning Model (KNN)
    model = KNeighborsClassifier(n_neighbors=5)
    accuracies = []

    for _ in range(20):
        model.fit(X_train, y_train)
        accuracy = accuracy_score(y_test, model.predict(X_test))
        accuracies.append(accuracy)

    return accuracies[-1]

## Function to Train Deep Model

In [158]:
@ray.remote
def train_deep_model(X_train, y_train, X_test, y_test):
    # Create and train the Deep Learning Model (DNN)
    model = create_deep_model(X_train.shape[1], len(np.unique(y_train)))

    for _ in range(20):
        model.fit(X_train, y_train, epochs=10, batch_size=10, verbose=0)

    # Evaluate the model on the test data and return the accuracy and weights
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    weights = model.get_weights()
    return accuracy, weights

# Functions to perform Federated Learning on DL and SL

## Peform FL on KNN

In [36]:
def fl_knn():
    # Generate some random numerical data
    dataset = pd.read_excel("DS_NSL_Final.xlsx")
    ipt_data = dataset.iloc[:, :-2]
    opt_data = dataset.iloc[:, -2:-1]
    x_train, x_test, y_train, y_test = train_test_split(ipt_data, opt_data, test_size=0.3, random_state=1985)

    # Reshape y_train and y_test into 1D arrays
    y_train = np.ravel(y_train)
    y_test = np.ravel(y_test)

    # Initialize Ray
    ray.shutdown()
    ray.init()

    num_clients = 10
    x_batches = np.array_split(x_train, num_clients)
    y_batches = np.array_split(y_train, num_clients)
    model = create_shallow_model()

    for round_ in range(4):
        # Train the model in parallel
        startTime = time.time()
        results = [train_shallow_model.remote(x_batches[i], y_batches[i], x_test, y_test) for i in range(num_clients)]
        accuracies = ray.get(results)
        print("Round:", round_+1, " Average Accuracy:", np.mean(accuracies), " Total time: ", time.time() - startTime, "seconds")
        
    # Fit the model with all the training data
    model.fit(x_train, y_train)

    accuracy = model.score(x_test, y_test)
    print("Test Accuracy:", accuracy)
    print("*********************** The value of Accuracy = ", accuracy, "  ********************************")
    ray.shutdown()

### Execute FL on SL (KNN)

In [37]:
fl_knn()

2023-05-27 17:13:33,748	INFO worker.py:1625 -- Started a local Ray instance.


Round: 1  Average Accuracy: 0.9158549396982811  Total time:  84.37151384353638 seconds
Round: 2  Average Accuracy: 0.9158549396982811  Total time:  80.81651782989502 seconds
Round: 3  Average Accuracy: 0.9158549396982811  Total time:  81.24969291687012 seconds
Round: 4  Average Accuracy: 0.9158549396982811  Total time:  81.79906272888184 seconds
Test Accuracy: 0.9548262386015044
*********************** The value of Accuracy =  0.9548262386015044   ********************************


## Perform FL on DL (DNN)

In [172]:
def fl_dnn():
    # Generate some random numerical data
    dataset = pd.read_excel("DS_NSL_Final.xlsx")
    ipt_data = dataset.iloc[:, :40]
    opt_data = dataset.iloc[:, 40:41]
    x_train, x_test, y_train, y_test = train_test_split(ipt_data, opt_data, test_size=0.3, random_state=95)

    # Reshape y_train and y_test into 1D arrays
    y_train = np.ravel(y_train)
    y_test = np.ravel(y_test)

    # Initialize Ray
    ray.shutdown()
    ray.init()

    num_clients = 10
    x_batches = np.array_split(x_train, num_clients)
    y_batches = np.array_split(y_train, num_clients)
    model = create_deep_model(x_train.shape[1], len(np.unique(y_train)))
    
    model.summary()

    for round_ in range(4):
        # Train the model in parallel
        startTime = time.time()
        results = [train_deep_model.remote(x_batches[i], y_batches[i], x_test, y_test) for i in range(num_clients)]
        accuracies, weights = zip(*ray.get(results))
        print("Round:", round_+1, " Average Accuracy:", np.mean(accuracies), " Total time:", time.time() - startTime, "seconds")

        # Combine the weights from each batch and update the model
        combined_weights = [np.mean(w, axis=1) for w in zip(*weights)]
        # Adjust the weights of the global model using federated weights
        global_weights = model.get_weights()

        # Combine the weights from each batch and update the model
        combined_weights = [np.mean(w, axis=0) for w in zip(*global_weights)]
        adjusted_weights = []
        for i, layer_weights in enumerate(combined_weights):
            expected_shape = model.layers[i].get_weights()[0].shape
            if layer_weights.shape != expected_shape:
                # Reshape the layer weights to match the expected shape
                if len(layer_weights.shape) == 2:  # Fully connected layer
                    layer_weights = layer_weights.mean(axis=1).reshape(expected_shape)
                elif len(layer_weights.shape) == 3:  # Convolutional layer
                    layer_weights = layer_weights.mean(axis=(0, 1)).reshape(expected_shape)
            adjusted_weights.append(layer_weights)

        # Ensure the number of adjusted weights matches the number of layers
        while len(adjusted_weights) < len(model.layers):
            adjusted_weights.append([])

        model.set_weights(adjusted_weights)
    
    # Fit the model with all the training data
    model.fit(x_train, y_train, epochs=10, batch_size=10, verbose=0)
    
    # Evaluate the model on the test data
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f'Test loss: {loss:.3f}, Test accuracy: {accuracy:.3f}')
    print("*********************** The value of Accuracy =", accuracy, "  ********************************")
    ray.shutdown()

### Execute FL on DL (DNN)

In [173]:
fl_dnn()

(train_deep_model pid=1299) 2023-05-29 14:38:46.071098: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA [repeated 7x across cluster]
(train_deep_model pid=1299) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags. [repeated 7x across cluster]
2023-05-29 15:00:35,405	INFO worker.py:1625 -- Started a local Ray instance.


Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_220 (Dense)           (None, 64)                2624      
                                                                 
 dense_221 (Dense)           (None, 32)                2080      
                                                                 
 dense_222 (Dense)           (None, 16)                528       
                                                                 
 dense_223 (Dense)           (None, 2)                 34        
                                                                 
Total params: 5,266
Trainable params: 5,266
Non-trainable params: 0
_________________________________________________________________


(pid=1698) 2023-05-29 15:00:37.351100: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
(pid=1698) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
(train_deep_model pid=1698) 2023-05-29 15:00:46.702213: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA [repeated 8x across cluster]
(train_deep_model pid=1698) To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags. [repeated 8x across cluster]


Round: 1  Average Accuracy: 0.9467243790626526  Total time: 493.97032713890076 seconds


/Users/abdulhameed/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


ValueError: operands could not be broadcast together with shapes (64,) (32,) 